In [23]:
import openai
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFDirectoryLoader
import os
from langchain.embeddings.openai import OpenAIEmbeddings
openai.api_key = os.getenv("OPENAI_API_Key")
print(os.getenv("OPENAI_API_Key"))

None


In [19]:
pdf_folder_path = './dateset1'
os.listdir(pdf_folder_path)

['(ASCE)CO.1943-7862.0001590.pdf', '(ASCE)CO.1943-7862.0001593.pdf']

In [20]:
loader = PyPDFDirectoryLoader("{}".format(pdf_folder_path))
date = loader.load()

In [21]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(date)
db = Chroma.from_documents(documents, OpenAIEmbeddings())

ValidationError: 1 validation error for OpenAIEmbeddings
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)